### Leer el archivo JSON usando el "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"


In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
persons_schema = StructType(fields=[
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
])

In [0]:
person_df = spark.read.schema(persons_schema).json(
    f"{bronze_folder_path}/{v_file_date}/person.json"
)

In [0]:
from pyspark.sql.functions import current_timestamp, lit, concat


In [0]:
persons_with_columns = person_df.withColumnsRenamed({
    'personId': 'person_id',
}).withColumns({
    'ingestion_date': current_timestamp(),
    'enviroment': lit("production"),
    'name': concat(person_df.personName.forename, lit(" "), person_df.personName.surname),
    'file_date': lit(v_file_date)
})


In [0]:
persons_final_df = persons_with_columns.drop('personName')

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.persons")
overwrite_partition(persons_final_df, "movie_silver", "persons", "file_date")


In [0]:
#persons_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/persons")
#persons_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.persons")
persons_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.persons")

In [0]:
%fs
ls /mnt/datamoviehistory/silver/persons

In [0]:
%sql

select file_date, count(1) 
from movie_silver.persons
group by file_date;